In [39]:
import os
import ee
import geemap
import pandas as pd
import dtale

Upgrade geemap to the latest version

In [10]:
# geemap.update_package()

In [11]:
Map_S5P = geemap.Map()
Map_S5P

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [12]:
# Create a common region of interest
roi = ee.Geometry.Polygon([[[-121.39140625, 35.650331512181495],
          [-121.39140625, 30.687240701629882],
          [-112.7341796875, 30.687240701629882],
          [-112.7341796875, 35.650331512181495]]])

In [13]:
# Data COllection for July
image_S5P = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2") \
    .filterBounds(roi) \
    .filterDate('2018-07-01', '2018-07-31') \
    .sort('ALGORITHM_VERSION') \
    .select('NO2_column_number_density') \
    .median() \
    .clip(roi)

vis_params_S5P = {
    'min': 0,
    'max': 0.0002,
    'bands': ['NO2_column_number_density'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map_S5P.addLayer(image_S5P, vis_params_S5P, "Sentinel-5P TROPOMI NO2")


image_LANDSAT8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-07-01', '2018-07-31') \
    .sort('CLOUD_COVER') \
    .select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol') \
    .median() \
    .clip(roi)

vis_params_L8 = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}


Map_S5P.addLayer(image_LANDSAT8, vis_params_L8, "Landsat-8")

In [29]:
ground_truth= image_LANDSAT8.addBands(image_S5P)

Download sample data

In [30]:
work_dir = os.path.expanduser('/Users/mdnhuda/Desktop/PARC IEM Data/Correlation for Month with Scatter')
in_shp = os.path.join(work_dir, 'LA_Shp.shp')

in_fc = geemap.shp_to_ee(in_shp)
Map_S5P.addLayer(in_fc, {}, 'Shape FIle Geometry')

Export pixel values as a shapefile

Export pixel values as a csv

proj = ee.Projection('EPSG:4326')
reproj_image = ground_truth.resample('bilinear').reproject(crs=proj, scale=30)


out_csv = os.path.join(work_dir, 'ground_truth_qgis.csv')
geemap.extract_values_to_points(in_fc, reproj_image, out_csv)

df=pd.read_csv('ground_truth_qgis.csv')

df=df.drop('system:index', axis=1)
df['Los Angele']=df['Los Angeles']

In [43]:
df['Los Angele']=df['Los Angeles']

KeyError: 'Los Angeles'

In [42]:
dtale.show(df)